# Model Evaluations

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import os

sns.set(rc={'figure.figsize':(14,7)})
series_map = pickle.load(open('pickled/series_map.pckl','rb'))
approach_map = {
    'Unadjusted':'orig',
    'IgnoreCOVID':'no_covid',
    'OutlierDetect':'with_outlier_detection'
}

pd.options.display.float_format = '{:.2%}'.format

In [2]:
future_actuals = pd.read_csv('data/Future Passengers.csv',parse_dates=['DATE'])
future_actuals

,DATE,HOU-Int,HOU-Dom,IAH-Dom,IAH-Int
0,2021-10-01,27153,498237,1375815,285498
1,2021-11-01,34242,495800,1356136,336156
2,2021-12-01,39101,477752,1331893,417561
3,2022-01-01,29084,387446,1036051,287998


In [3]:
def compare_all_best_models_total_accr(series_map):
    """
    """
    all_results = pd.DataFrame(columns=approach_map.keys(),index=series_map.keys())
    all_best_models = all_results.copy()
    for series in series_map.keys():
        for apprt, appr in approach_map.items():
            results = pd.read_excel(os.path.join('fcst_results',
                                       '{}_series_{}_results.xlsx'.format(appr,series)),
                                       sheet_name=['lvl_fcsts','model_summaries'])
            best_model = results['model_summaries'].iloc[0,0]
            results_lvl = results['lvl_fcsts']
            results_lvl['DATE'] = pd.to_datetime(results_lvl['DATE'])
            results_lvl = results_lvl.loc[results_lvl['DATE'].isin(future_actuals['DATE'])].reset_index(drop=True)
            all_results.loc[series,apprt] = 1 - np.abs((future_actuals[series].sum() - results_lvl[best_model].sum()) / future_actuals[series].sum())
            all_best_models.loc[series,apprt] = best_model
    return all_results, all_best_models
        
def compare_all_best_models_mean_accr(series_map):
    """
    """
    all_results = pd.DataFrame(columns=approach_map.keys(),index=series_map.keys())
    all_best_models = all_results.copy()
    for series in series_map.keys():
        for apprt, appr in approach_map.items():
            results = pd.read_excel(os.path.join('fcst_results',
                                       '{}_series_{}_results.xlsx'.format(appr,series)),
                                       sheet_name=['lvl_fcsts','model_summaries'])
            best_model = results['model_summaries'].iloc[0,0]
            results_lvl = results['lvl_fcsts']
            results_lvl['DATE'] = pd.to_datetime(results_lvl['DATE'])
            results_lvl = results_lvl.loc[results_lvl['DATE'].isin(future_actuals['DATE'])].reset_index(drop=True)
            all_results.loc[series,apprt] = 1 - np.mean(np.abs((future_actuals[series] - results_lvl[best_model]) / future_actuals[series]))
            all_best_models.loc[series,apprt] = best_model
    return all_results, all_best_models

def compare_all_models_total_accr(series_map,model_type):
    """
    """
    best_model = model_type
    all_results = pd.DataFrame(columns=approach_map.keys(),index=series_map.keys())
    for series in series_map.keys():
        for apprt, appr in approach_map.items():
            results_lvl = pd.read_excel(os.path.join('fcst_results',
                                       '{}_series_{}_results.xlsx'.format(appr,series)),
                                       sheet_name='lvl_fcsts',parse_dates=['DATE'])
            results_lvl = results_lvl.loc[results_lvl['DATE'].isin(future_actuals['DATE'])]
            try:
                all_results.loc[series,apprt] = 1 - np.abs((future_actuals[series].sum() - results_lvl[best_model].sum()) / future_actuals[series].sum())
            except KeyError:
                pass
    return all_results.dropna(axis=1)

## Compare all best models - total accuracy

In [4]:
best_models_results = compare_all_best_models_total_accr(series_map)
best_models_results[0]

,Unadjusted,IgnoreCOVID,OutlierDetect
IAH-Dom,90.42%,87.52%,80.68%
IAH-Int,95.14%,54.88%,91.00%
HOU-Dom,97.00%,72.47%,99.19%
HOU-Int,99.42%,77.30%,90.24%


In [5]:
best_models_results[1]

,Unadjusted,IgnoreCOVID,OutlierDetect
IAH-Dom,weighted_ensemble,arima,prophet
IAH-Int,arima,prophet,arima
HOU-Dom,arima,weighted_ensemble,arima
HOU-Int,arima,lstm,arima


## Compare all best models - mean accuracy per month

In [6]:
best_models_mean_results = compare_all_best_models_mean_accr(series_map)
best_models_mean_results[0]

,Unadjusted,IgnoreCOVID,OutlierDetect
IAH-Dom,87.27%,86.26%,81.38%
IAH-Int,93.52%,52.53%,91.50%
HOU-Dom,93.47%,71.52%,94.29%
HOU-Int,92.74%,74.48%,90.53%


In [7]:
best_models_mean_results[1]

,Unadjusted,IgnoreCOVID,OutlierDetect
IAH-Dom,weighted_ensemble,arima,prophet
IAH-Int,arima,prophet,arima
HOU-Dom,arima,weighted_ensemble,arima
HOU-Int,arima,lstm,arima


## Compare all ARIMA models

In [8]:
compare_all_models_total_accr(series_map,'arima')

,Unadjusted,IgnoreCOVID,OutlierDetect
IAH-Dom,97.08%,87.52%,96.45%
IAH-Int,95.14%,58.56%,91.00%
HOU-Dom,97.00%,73.51%,99.19%
HOU-Int,99.42%,99.00%,90.24%


## Compare all Prophet models

In [9]:
compare_all_models_total_accr(series_map,'prophet')

,Unadjusted,IgnoreCOVID,OutlierDetect
IAH-Dom,80.52%,87.66%,80.68%
IAH-Int,81.41%,54.88%,81.51%
HOU-Dom,93.32%,70.56%,93.40%
HOU-Int,70.00%,69.69%,70.00%


## Compare all LSTM models

In [10]:
compare_all_models_total_accr(series_map,'lstm')

,Unadjusted,IgnoreCOVID
IAH-Dom,97.53%,94.62%
IAH-Int,92.04%,72.78%
HOU-Dom,87.45%,76.93%
HOU-Int,97.85%,77.30%


## Compare all Ensemble models

In [11]:
compare_all_models_total_accr(series_map,'weighted_ensemble')

,Unadjusted,IgnoreCOVID,OutlierDetect
IAH-Dom,90.42%,89.50%,82.59%
IAH-Int,95.20%,57.38%,90.21%
HOU-Dom,96.59%,72.47%,99.80%
HOU-Int,89.59%,88.90%,88.56%
